In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ghsperson-house/Person files/Person files/General Household Survey 2015 (House file).pdf
/kaggle/input/ghsperson-house/Person files/Person files/AJAD_2011_9_2_5Pede.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2017-PERSON/GHS-2017-PERSON.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2017-PERSON/GHS-2017-PERSON_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2018-PERSON/GHS-2018-PERSON_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2018-PERSON/GHS-2018-PERSON.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2015-PERSON/GHS-2015-PERSON.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2015-PERSON/GHS-2015-PERSON_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-Person-2014-rwt2018/GHS-Person-2014-rwt2018_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-Person-2014-rwt2018/GHS-Person-2014-rwt2018.pdf
/kaggle/input/ghsperson-house/Person 

In [63]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.utils import resample
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

In [64]:
#import the household file data
house2011 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2011-HOUSE_F1.csv')
house2012 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2012-HOUSE_F1.csv')
house2013 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2013-HOUSE_F1.csv')
house2014 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2014-HOUSE_F1.csv')
house2015 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2015-HOUSE_F1.csv')
house2016 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2016-HOUSE_F1.csv')
house2017 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2017-HOUSE_F1.csv')
house2018 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2018-HOUSE_F1.csv')

In [65]:
#import the person file data
person2011 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-Person-2011/GHS-Person-2011_F1.csv')
person2012 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2012-PERSON/GHS-2012-PERSON_F1.csv')
person2013 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2013-PERSON (1)/GHS-2013-PERSON_F1.csv')
person2014 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-Person-2014-rwt2018/GHS-Person-2014-rwt2018_F1.csv')
person2015 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2015-PERSON/GHS-2015-PERSON_F1.csv')
person2016 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2016-PERSONs/GHS-2016-PERSONs_F1.csv')
person2017 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2017-PERSON/GHS-2017-PERSON_F1.csv')
person2018 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2018-PERSON/GHS-2018-PERSON_F1.csv')

In [66]:
def rename_columns(df):
    return df.rename(columns = {'UqNr': 'uqnr', 'PersonNR': 'personnr','Personnr': 'personnr', 'Q16hiedu':'Q15HIEDU','Q11relsh':'Q11RELSH','q11RELSH':'Q11RELSH'})

In [67]:
def get_renamed_dfs(dfs):
    new_dfs = []
    for df in dfs:
        new_df = rename_columns(df)
        new_dfs.append(new_df)
    return new_dfs

In [68]:
house_dfs = [house2011, house2012, house2013, house2014, house2015, house2016, house2017, house2018]
renamed_house_dfs = get_renamed_dfs(house_dfs)
house_file_data = pd.concat(renamed_house_dfs)
print(house_file_data.columns)

Index(['uqnr', 'personnr', 'PSU', 'Prov', 'head_popgrp', 'head_sex',
       'head_age', 'Q31MainD', 'Q31OthrD', 'Q32roof',
       ...
       'Q82aNAgri', 'Q82bNpersno', 'Q83grai', 'Q83fruit', 'Q87Chickens',
       'Q88dType', 'Q815bTotveh', 'Q528cFBE', 'Q66bfixedint', 'Q66bmobint'],
      dtype='object', length=977)


In [69]:
person_dfs = [person2011, person2012, person2013, person2014, person2015, person2016, person2017, person2018]
renamed_person_dfs = get_renamed_dfs(person_dfs)
person_file_data = pd.concat(renamed_person_dfs )
print(person_file_data.columns)

Index(['uqnr', 'personnr', 'PSU', 'Prov', 'B_Nights', 'Gender', 'Age',
       'Age_grp', 'Race', 'Q11RELSH',
       ...
       'Q18STORIES', 'Q18SING', 'Q18DRAW', 'Q18TALK', 'Q18COUNT', 'Q18NAME',
       'Q18PLAY', 'Q23MAL', 'Q26MAL', 'Prov_add_1588231763189'],
      dtype='object', length=499)


In [70]:
# house_file_data.to_csv('house_file_data.csv')
# person_file_data.to_csv('person_file_data.csv')

In [71]:
# person_file_data = person_file_data.iloc[0:10000]
# house_file_data = house_file_data.iloc[0:10000]

In [72]:
print('person_rows: {:}'.format(person_file_data.shape[0]))
print('person_columns: {:}'.format(person_file_data.shape[1]))
print("\n")
print('house_rows: {:}'.format(house_file_data.shape[0]))
print('house_columns: {:}'.format(house_file_data.shape[1]))

person_rows: 661980
person_columns: 499


house_rows: 186517
house_columns: 977


In [73]:
house_data_chosen_columns = ['uqnr', 'personnr','totmhinc', 'hholdsz', 'head_age', 'head_sex']
person_data_chosen_columns = ['uqnr', 'personnr','employ_Status1','employ_Status2','Q15HIEDU', 'Q11RELSH']

In [74]:
house_data_chosen_subset = house_file_data[house_data_chosen_columns]
house_data_chosen_subset.head()

,uqnr,personnr,totmhinc,hholdsz,head_age,head_sex
0,101000190000001601,1,7400,4,52,2
1,101000190000005601,1,3280,4,50,1
2,101000190000007601,1,2650,5,45,1
3,101000190000011601,1,4716,6,59,1
4,101000240000001401,1,1600,2,48,1


In [75]:
house_data_chosen_subset.isnull().sum()

uqnr        0
personnr    0
totmhinc    0
hholdsz     0
head_age    0
head_sex    0
dtype: int64

In [76]:
#Drop missing columns if they are less than 30% of the data 
final_house_data = house_data_chosen_subset.dropna()

In [77]:
final_house_data.isnull().sum()

uqnr        0
personnr    0
totmhinc    0
hholdsz     0
head_age    0
head_sex    0
dtype: int64

In [78]:
house_data_chosen_subset.groupby('uqnr').sum()

,personnr,totmhinc,hholdsz,head_age,head_sex
uqnr,,,,,
101000170000001501,3,33000,8,96,3
101000170000005701,3,5720,6,134,3
101000170000009901,3,4700,7,158,4
101000170000012001,1,4900,3,58,1
101000170000014101,1,1460,1,41,1
...,...,...,...,...,...
987106510000014601,2,5960,6,59,2
987106510000016101,4,62288,21,187,5
987106510000016102,1,2000,1,53,2


In [79]:
person_data_chosen_subset = person_file_data[person_data_chosen_columns]
person_data_chosen_subset.head()

,uqnr,personnr,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
0,101000190000001601,1,NaN,NaN,12.0,1
1,101000190000001601,2,NaN,NaN,12.0,3
2,101000190000001601,3,NaN,NaN,12.0,9
3,101000190000001601,4,NaN,NaN,98.0,7
4,101000190000005601,1,NaN,NaN,7.0,1


In [80]:
person_data_chosen_subset.isnull().sum()

uqnr                   0
personnr               0
employ_Status1    279042
employ_Status2    279042
Q15HIEDU           93749
Q11RELSH               0
dtype: int64

In [81]:
person_data_chosen_subset.isnull().sum()['employ_Status1'] + person_data_chosen_subset.isnull().sum()['employ_Status2']

558084

In [82]:
#Drop missing columns if they are less than 30% of the data 
final_person_data = person_data_chosen_subset.dropna()

In [83]:
final_person_data.isnull().sum()

uqnr              0
personnr          0
employ_Status1    0
employ_Status2    0
Q15HIEDU          0
Q11RELSH          0
dtype: int64

In [84]:
person_data_chosen_subset.groupby('uqnr').sum()

,personnr,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
uqnr,,,,,
101000170000001501,15,10.0,10.0,173.0,15
101000170000005701,9,10.0,4.0,28.0,9
101000170000009901,12,18.0,12.0,19.0,16
101000170000012001,6,0.0,0.0,110.0,19
101000170000014101,1,0.0,0.0,5.0,1
...,...,...,...,...,...
987106510000014601,13,37.0,25.0,152.0,12
987106510000016101,72,127.0,79.0,388.0,69
987106510000016102,1,1.0,1.0,6.0,1


In [85]:
print('final_person_rows: {:}'.format(final_person_data.shape[0]))
print('final_person_columns: {:}'.format(final_person_data.shape[1]))
print("\n")
print('final_house_rows: {:}'.format(final_house_data.shape[0]))
print('final_house_columns: {:}'.format(final_house_data.shape[1]))

final_person_rows: 382938
final_person_columns: 6


final_house_rows: 186517
final_house_columns: 6


In [86]:
def get_emp_status(series):
    status = None
    for value in series:
        if value == 1:
            status = "employed"
            break
        else:
            status = "unemployed"
            continue
    return status

In [87]:
def find_head_employment_status(house_id):
    iden = final_person_data[final_person_data['uqnr'] == house_id] 
    emp_status = None
    chosen_series = iden['Q11RELSH']
    for value in chosen_series:
        if value == 1:
            emp_status_series1 = iden['employ_Status1']
            emp_status1 = get_emp_status(emp_status_series1)
            emp_status_series2 = iden['employ_Status2']
            emp_status2 = get_emp_status(emp_status_series2)
#             emp_status = emp_status1
            if (emp_status1 == "employed") or (emp_status2 == "employed"):
                emp_status = 1
            else:
                emp_status = 0
        
    return emp_status

In [88]:
empl_status = find_head_employment_status(101000170000001501)
print(empl_status)

1


In [89]:
# final_house_data_subset = pd.DataFrame()
# final_house_data['status'] = final_house_data['uqnr'].apply(find_head_employment_status)
# final_house_data.head()

In [90]:
left = final_house_data.set_index(['uqnr', 'personnr'])
right = final_person_data.set_index(['uqnr', 'personnr'])

final = left.join(right, lsuffix='_house', rsuffix='_person')
final.head()

totmhinc  hholdsz  head_age  head_sex  \
uqnr               personnr                                          
101000170000001501 1            10000        2        31         1   
                   1            10000        3        32         1   
                   1            13000        3        33         1   
101000170000005701 1              720        2        43         1   
                   1             3200        2        45         1   

                             employ_Status1  employ_Status2  Q15HIEDU  \
uqnr               personnr                                             
101000170000001501 1                    1.0             1.0      12.0   
                   1                    1.0             1.0      12.0   
                   1                    1.0             1.0      12.0   
101000170000005701 1                    1.0             1.0      10.0   
                   1                    1.0             1.0      10.0   

                             Q11RELSH  
uqnr               personnr            
101000170000001501 1              1.0  
                   1              1.0  
                   1              1.0  
101000170000005701 1              1.0  
                   1              1.0

In [91]:
# house_emp_statuses = []
# for house_id in final_house_data['uqnr']:
#     empl_status = None
#     empl_status = find_head_employment_status(final_person_data, house_id)
# #     print(empl_status)
#     house_emp_statuses.append(empl_status)

In [92]:
# final_house_data['status'] = house_emp_statuses
# final_house_data.head()

In [93]:
final_house_data_subset.isnull().sum()

NameError: name 'final_house_data_subset' is not defined

In [ ]:
# house_emp_status = find_head_employment_status(final_person_data, 101000170000005701)
# print(house_emp_status)

In [ ]:
def find_head_maritial_status(data, house_id):
    return 0

In [ ]:
def find_head_education_level(data, house_id):
    return 0